In [1]:
import Pkg; Pkg.status()
include("../src/Arthur.jl")
using RigidBodyDynamics
using StaticArrays
using RobotDynamics
using LinearAlgebra
using ForwardDiff, FiniteDiff
using SparseArrays
using OSQP
using RobotDynamics
using Statistics
using ControlSystems

      Status `~/catkin_workspace/src/ros_kortex/kortex_control/mpc/notebooks/Project.toml` (empty project)


In [2]:
const dt = 0.05  # time step (s)
const tf = 5   # time horizon (s)
const N = Int(round(tf / dt)) + 1

101

In [67]:
model = Arthur()
const n = state_dim(model)
const m = control_dim(model)
const xeq = [-1.3100000000000003, 0.47762070845504206, 2.955910662235214, -1.9320918417907373, -1.4414028267565981, 1.568570418791257, 2.312454362330413, 0, 0, 0, 0, 0, 0, 0]
state = MechanismState(model.mechanism)
v̇ = similar(velocity(state))
for joint in joints(state.mechanism)
    if joint != joints(state.mechanism)[1] && joint != joints(state.mechanism)[end]
        v̇[joint][1] = 0.0
    end
end
zero!(state)
set_configuration!(state, xeq[1:7])
set_velocity!(state, xeq[8:end])
const ueq = inverse_dynamics(state, v̇)
norm(dynamics(model, xeq, ueq)) ≈ 0 # make sure it's an equilibrium point

Q = Diagonal(1 * ones(n))
R = Diagonal(ones(m))

7×7 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

In [68]:
# Evaluate the continuous and discrete Jacobians
zeq = KnotPoint(xeq,ueq,dt)   # create a `KnotPoint` type that stores everything together
∇f = RobotDynamics.DynamicsJacobian(model)
jacobian!(∇f, model, zeq)
discrete_jacobian!(RK4, ∇f, model, zeq)

# Extract pieces of the Jacobian
const A = ∇f.A
const B = ∇f.B;

In [69]:
qref = typeof(zeros(7))[]
push!(qref,  [4.9268856741946365e-05, 0.26016423071338934, 3.140069344584098, -2.27008100955155, -2.40769461674617e-05, 0.9598460188468598, 1.569995694393751])
push!(qref,  [-0.003768053224579125, 0.26079787208394173, 3.1395327288847428, -2.269096150780195, -0.004224076946167462, 0.9616197667116058, 1.5721591274301712])
push!(qref,  [-0.01522001946854234, 0.2626987961955988, 3.137922881786677, -2.2661415744661304, -0.016824076946167464, 0.9669410103058437, 1.578649426539432])
push!(qref,  [-0.03430662987514771, 0.26586700304836064, 3.135239803289901, -2.2612172806093556, -0.03782407694616747, 0.9758097496295736, 1.5894665917215332])
push!(qref,  [-0.0610278844443952, 0.2703024926422273, 3.131483493394414, -2.254323269209871, -0.06722407694616747, 0.9882259846827953, 1.6046106229764747])
push!(qref,  [-0.0953296828066516, 0.2759962847990445, 3.126661557197997, -2.2454734980167035, -0.10496455313664366, 1.0041645773154986, 1.6240508594052145])
push!(qref,  [-0.13259401741002397, 0.2821818315115795, 3.121423165847148, -2.23585940048681, -0.14596455313664364, 1.0214797350427807, 1.6451700866655072])
push!(qref,  [-0.1698583520133963, 0.2883673782241145, 3.116184774496299, -2.226245302956917, -0.18696455313664362, 1.0387948927700625, 1.6662893139258002])
push!(qref,  [-0.20712268661676866, 0.2945529249366496, 3.1109463831454507, -2.2166312054270234, -0.2279645531366436, 1.0561100504973449, 1.6874085411860928])
push!(qref,  [-0.24438702122014103, 0.3007384716491846, 3.1057079917946018, -2.20701710789713, -0.2689645531366436, 1.0734252082246267, 1.7085277684463855])
push!(qref,  [-0.2816513558235134, 0.3069240183617196, 3.100469600443753, -2.1974030103672364, -0.3099645531366436, 1.090740365951909, 1.7296469957066785])
push!(qref,  [-0.3189156904268857, 0.31310956507425464, 3.095231209092904, -2.1877889128373433, -0.3509645531366436, 1.108055523679191, 1.7507662229669712])
push!(qref,  [-0.3561800250302581, 0.3192951117867897, 3.089992817742055, -2.17817481530745, -0.39196455313664363, 1.1253706814064732, 1.7718854502272638])
push!(qref,  [-0.3934443596336305, 0.3254806584993247, 3.0847544263912066, -2.1685607177775563, -0.43296455313664367, 1.1426858391337553, 1.7930046774875568])
push!(qref,  [-0.43070869423700286, 0.33166620521185974, 3.0795160350403576, -2.158946620247663, -0.47396455313664365, 1.1600009968610374, 1.8141239047478495])
push!(qref,  [-0.4679730288403753, 0.3378517519243948, 3.0742776436895087, -2.1493325227177693, -0.5149645531366437, 1.1773161545883195, 1.8352431320081424])
push!(qref,  [-0.5052373634437477, 0.34403729863692983, 3.06903925233866, -2.1397184251878762, -0.5559645531366437, 1.1946313123156016, 1.856362359268435])
push!(qref,  [-0.54250169804712, 0.3502228453494649, 3.063800860987811, -2.1301043276579827, -0.5969645531366438, 1.2119464700428837, 1.8774815865287278])
push!(qref,  [-0.5797660326504925, 0.3564083920619999, 3.0585624696369624, -2.1204902301280892, -0.6379645531366438, 1.2292616277701658, 1.8986008137890207])
push!(qref,  [-0.6170303672538648, 0.36259393877453494, 3.0533240782861135, -2.1108761325981957, -0.6789645531366438, 1.2465767854974479, 1.9197200410493134])
push!(qref,  [-0.6542947018572371, 0.36877948548706996, 3.0480856869352646, -2.1012620350683022, -0.7199645531366438, 1.26389194322473, 1.940839268309606])
push!(qref,  [-0.6915590364606096, 0.37496503219960503, 3.0428472955844157, -2.091647937538409, -0.7609645531366438, 1.281207100952012, 1.961958495569899])
push!(qref,  [-0.728823371063982, 0.38115057891214005, 3.0376089042335668, -2.0820338400085157, -0.8019645531366438, 1.2985222586792942, 1.983077722830192])
push!(qref,  [-0.7660877056673544, 0.3873361256246751, 3.0323705128827183, -2.072419742478622, -0.8429645531366439, 1.3158374164065763, 2.0041969500904844])
push!(qref,  [-0.8033520402707267, 0.39352167233721014, 3.0271321215318694, -2.0628056449487286, -0.8839645531366439, 1.3331525741338583, 2.0253161773507773])
push!(qref,  [-0.8406163748740991, 0.39970721904974515, 3.0218937301810205, -2.053191547418835, -0.9249645531366439, 1.3504677318611407, 2.04643540461107])
push!(qref,  [-0.8778807094774715, 0.4058927657622802, 3.0166553388301716, -2.043577449888942, -0.965964553136644, 1.3677828895884225, 2.067554631871363])
push!(qref,  [-0.915145044080844, 0.41207831247481524, 3.0114169474793226, -2.0339633523590486, -1.006964553136644, 1.3850980473157049, 2.0886738591316556])
push!(qref,  [-0.9524093786842164, 0.4182638591873503, 3.006178556128474, -2.024349254829155, -1.0479645531366442, 1.402413205042987, 2.1097930863919485])
push!(qref,  [-0.9896737132875887, 0.42444940589988533, 3.0009401647776253, -2.0147351572992616, -1.088964553136644, 1.419728362770269, 2.1309123136522414])
push!(qref,  [-1.026938047890961, 0.43063495261242035, 2.9957017734267763, -2.005121059769368, -1.1299645531366442, 1.4370435204975511, 2.1520315409125343])
push!(qref,  [-1.0642023824943334, 0.43682049932495537, 2.9904633820759274, -1.9955069622394748, -1.1709645531366442, 1.4543586782248332, 2.173150768172827])
push!(qref,  [-1.101466717097706, 0.44300604603749044, 2.9852249907250785, -1.9858928647095815, -1.2119645531366443, 1.4716738359521153, 2.1942699954331197])
push!(qref,  [-1.1387310517010785, 0.4491915927500255, 2.97998659937423, -1.976278767179688, -1.2529645531366442, 1.4889889936793974, 2.215389222693412])
push!(qref,  [-1.1759953863044506, 0.45537713946256053, 2.974748208023381, -1.9666646696497945, -1.2939645531366444, 1.5063041514066795, 2.236508449953705])
push!(qref,  [-1.213259720907823, 0.46156268617509555, 2.969509816672532, -1.9570505721199012, -1.3349645531366443, 1.5236193091339616, 2.257627677213998])
push!(qref,  [-1.2478021278200226, 0.46729641720217435, 2.9646540572035307, -1.9481387245716801, -1.3729697583408722, 1.5396697023673502, 2.277204276379416])
push!(qref,  [-1.2748022884966306, 0.4717782027187738, 2.9608585403999896, -1.9411727561509207, -1.402676624155692, 1.5522155332738579, 2.292506375168055])
push!(qref,  [-1.2941678050105965, 0.47499270549426853, 2.9581362549951593, -1.9361765052728712, -1.423983489970512, 1.5612138684508734, 2.3034816078838536])
push!(qref,  [-1.3058986773619201, 0.4769399255286585, 2.9564872009890393, -1.9331499719375316, -1.4368903557853316, 1.566664707898397, 2.310129974526812])
push!(qref,  [-1.3099949055506017, 0.47761986282194374, 2.95591137838163, -1.932093156144902, -1.4413972216001514, 1.5685680516164289, 2.3124514750969296])
push!(qref,  [-1.3100000000000003, 0.47762070845504206, 2.955910662235214, -1.9320918417907373, -1.4414028267565981, 1.568570418791257, 2.312454362330413]);

In [70]:
q̇ref = typeof(zeros(7))[]
push!(q̇ref,  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
push!(q̇ref,  [-0.07634644162642143, 0.012672827411047383, -0.010732313987104905, 0.019697175427098694, -0.084, 0.03547495729491942, 0.04326866072840471])
push!(q̇ref,  [-0.15269288325284286, 0.025345654822094766, -0.02146462797420981, 0.03939435085419739, -0.168, 0.07094991458983883, 0.08653732145680942])
push!(q̇ref,  [-0.22903932487926432, 0.03801848223314215, -0.032196941961314715, 0.05909152628129609, -0.252, 0.10642487188475827, 0.12980598218521414])
push!(q̇ref,  [-0.3053857665056857, 0.05069130964418953, -0.04292925594841962, 0.07878870170839478, -0.336, 0.14189982917967767, 0.17307464291361885])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3726433460337236, 0.061855467125350325, -0.05238391350848822, 0.09614097529893409, -0.41, 0.17315157727282096, 0.21119227260292772])
push!(q̇ref,  [-0.3081748275792907, 0.05115426887151827, -0.04332132502896183, 0.07950827192114293, -0.3390686581481977, 0.14319578771253505, 0.17465531825059288])
push!(q̇ref,  [-0.2318283859528692, 0.038481441460470875, -0.03258901104185691, 0.05981109649404422, -0.2550686581481976, 0.1077208304176156, 0.13138665752218814])
push!(q̇ref,  [-0.1554819443264477, 0.02580861404942348, -0.021856697054752, 0.04011392106694551, -0.17106865814819755, 0.07224587312269615, 0.08811799679378339])
push!(q̇ref,  [-0.07913550270002623, 0.013135786638376092, -0.011124383067647084, 0.0204167456398468, -0.08706865814819748, 0.036770915827776696, 0.04484933606537864])
push!(q̇ref,  [-0.002789061073605048, 0.00046295922732875194, -0.00039206908054222056, 0.0007195702127481829, -0.0030686581481977893, 0.0012959585328574108, 0.0015806753369740911])
push!(q̇ref,  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]);

In [71]:
function nominal_trajectory(x0,N,dt)
    Xref = [fill(NaN, length(x0)) for k = 1:N]
    Xref[1] = x0
    # TODO: Design a trajectory that linearly interpolates from x0 to the origin
    for k = 2:N
        Xref[k][1:7] = x0[1:7] - (k-1)*(x0[1:7] - xeq[1:7])/(N-1)
        if k != 2
            Xref[k-1][8:14] = (Xref[k][1:7] - Xref[k-1][1:7])/dt
        end
    end
    
    Xref[N][8:14] = zeros(7)
    
    return SVector{14}.(Xref)
end

nominal_trajectory (generic function with 1 method)

In [72]:
# Start 500m off the ground and 200m West
x0_ref = [4.9268856741946365e-05, 0.26016423071338934, 3.140069344584098, -2.27008100955155, -2.40769461674617e-05, 0.9598460188468598, 1.569995694393751, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# Generate reference trajectory
Xref = nominal_trajectory(x0_ref,N,dt)
# Xref = typeof(zeros(14))[]
# for k = 1:N
#     push!(Xref, [qref[k]; q̇ref[k]])
# end
Uref = [copy(ueq) for k = 1:N]
tref = range(0, tf, length=N);
print(Xref)

SVector{14, Float64}[[4.9268856741946365e-5, 0.26016423071338934, 3.140069344584098, -2.27008100955155, -2.40769461674617e-5, 0.9598460188468598, 1.569995694393751, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.013051223831825476, 0.26233879549080585, 3.1382277577606095, -2.266701117873942, -0.014437864444271769, 0.9659332628463038, 1.5774202810731175, -0.26200985377134844, 0.04349129554833131, -0.03683173646978233, 0.06759783355216875, -0.2882757499620861, 0.12174487998888095, 0.1484917335873348], [-0.026151716520392897, 0.2645133602682224, 3.1363861709371204, -2.2633212261963336, -0.028851651942376075, 0.9720205068457478, 1.5848448677524842, -0.2620098537713484, 0.0434912955483302, -0.036831736469773446, 0.06759783355215987, -0.288275749962086, 0.12174487998887873, 0.14849173358733037], [-0.03925220920896032, 0.26668792504563893, 3.1345445841136317, -2.2599413345187256, -0.04326543944048038, 0.9781077508451917, 1.5922694544318508, -0.26200985377134844, 0.0434912955483302, -0.03683173646978

 -0.5045066393798182, 1.172899558827399, 1.8298562281715827, -0.2620098537713489, 0.0434912955483302, -0.03683173646978233, 0.06759783355216875, -0.2882757499620858, 0.12174487998887873, 0.14849173358733037], [-0.4715684679316853, 0.3384485627003843, 3.0737722189384997, -2.1484049091576574, -0.5189204268779225, 1.1789868028268429, 1.8372808148509492, -0.2620098537713489, 0.04349129554833131, -0.036831736469773446, 0.06759783355215987, -0.2882757499620858, 0.12174487998887873, 0.1484917335873348], [-0.4846689606202527, 0.3406231274778009, 3.071930632115011, -2.1450250174800494, -0.5333342143760268, 1.1850740468262868, 1.844705401530316, -0.26200985377134667, 0.0434912955483302, -0.036831736469773446, 0.06759783355215987, -0.2882757499620858, 0.12174487998887873, 0.1484917335873348], [-0.49776945330882005, 0.3427976922552174, 3.0700890452915224, -2.1416451258024414, -0.5477480018741311, 1.1911612908257307, 1.8521299882096827, -0.26200985377135, 0.0434912955483302, -0.03683173646978233, 0

 0.0434912955483302, -0.036831736469773446, 0.06759783355215987, -0.2882757499620858, 0.12174487998887873, 0.1484917335873348], [-0.930085712031545, 0.4145583299099628, 3.0093166801163904, -2.030108700441373, -1.0234029893115733, 1.3920403428073818, 2.097141348628781, -0.26200985377135, 0.0434912955483302, -0.036831736469773446, 0.06759783355216875, -0.2882757499620858, 0.12174487998888317, 0.14849173358732592], [-0.9431862047201125, 0.4167328946873793, 3.0074750932929017, -2.0267288087637647, -1.0378167768096775, 1.398127586806826, 2.1045659353081474, -0.2620098537713478, 0.0434912955483302, -0.03683173646978233, 0.06759783355215987, -0.2882757499620858, 0.12174487998887873, 0.1484917335873348], [-0.9562866974086799, 0.4189074594647958, 3.0056335064694126, -2.0233489170861567, -1.0522305643077818, 1.4042148308062699, 2.111990521987514, -0.26200985377135, 0.04349129554833131, -0.036831736469773446, 0.06759783355215987, -0.2882757499620858, 0.12174487998887873, 0.1484917335873348], [-0.

In [73]:
# All of the code in this cell is provided. There is no need to modify it.
"""
    MPCController

An MPC controller that uses a solver of type `S` to solve a QP at every iteration.

It will track the reference trajectory specified by `Xref`, `Uref` and `times` 
with an MPC horizon of `Nmpc`. It will track the terminal reference state if 
the horizon extends beyond the reference horizon.
"""
struct MPCController{S}
    P::SparseMatrixCSC{Float64,Int}
    q::Vector{Float64}
    A::SparseMatrixCSC{Float64,Int}
    lb::Vector{Float64}
    ub::Vector{Float64}
    Nmpc::Int
    solver::S
    Xref::Vector{Vector{Float64}}
    Uref::Vector{Vector{Float64}}
    times::Vector{Float64}
end

"""
    OSQPController(n,m,N,Nref,Nd)

Generate an `MPCController` that uses OSQP to solve the QP.
Initializes the controller with matrices consistent with `n` states,
`m` controls, and an MPC horizon of `N`, and `Nd` constraints. 

Use `Nref` to initialize a reference trajectory whose length may differ from the 
horizon length.
"""
function OSQPController(n::Integer, m::Integer, N::Integer, Nref::Integer=N, Nd::Integer=(N-1)*n)
    Np = (N-1)*(n+m)   # number of primals
    P = spzeros(Np,Np)
    q = zeros(Np)
    A = spzeros(Nd,Np)
    lb = zeros(Nd)
    ub = zeros(Nd)
    Xref = [zeros(n) for k = 1:Nref]
    Uref = [zeros(m) for k = 1:Nref]
    tref = zeros(Nref)
    solver = OSQP.Model()
    MPCController{OSQP.Model}(P,q, A,lb,ub, N, solver, Xref, Uref, tref)
end

isconstrained(ctrl::MPCController) = length(ctrl.lb) != (ctrl.Nmpc - 1) * length(ctrl.Xref[1])

"""
    buildQP!(ctrl, A,B,Q,R,Qf; kwargs...)

Build the QP matrices `P` and `A` for the MPC problem. Note that these matrices
should be constant between MPC iterations.

Any keyword arguments will be passed to `initialize_solver!`.
"""
function buildQP!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    if isconstrained(ctrl)
        buildQP_constrained!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    else
        buildQP_unconstrained!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    end
end

"""
    updateQP!(ctrl::MPCController, x, time)

Update the vectors in the QP problem for the current state `x` and time `time`.
This should update `ctrl.q`, `ctrl.lb`, and `ctrl.ub`.
"""
function updateQP!(ctrl::MPCController, x, time)
    if isconstrained(ctrl)
        updateQP_constrained!(ctrl, x, time)
    else
        updateQP_unconstrained!(ctrl, x, time)
    end
end


"""
    initialize_solver!(ctrl::MPCController; kwargs...)

Initialize the internal solver once the QP matrices are initialized in the 
controller.
"""
function initialize_solver!(ctrl::MPCController{OSQP.Model}; tol=1e-6, verbose=false)
    OSQP.setup!(ctrl.solver, P=ctrl.P, q=ctrl.q, A=ctrl.A, l=ctrl.lb, u=ctrl.ub, 
        verbose=verbose, eps_rel=tol, eps_abs=tol, polish=1)
end

"""
    get_control(ctrl::MPCController, x, t)

Get the control from the MPC solver by solving the QP. 
If you want to use your own QP solver, you'll need to change this
method.
"""
function get_control(ctrl::MPCController{OSQP.Model}, x, time)
    # Update the QP
    updateQP!(ctrl, x, time)
    OSQP.update!(ctrl.solver, q=ctrl.q, l=ctrl.lb, u=ctrl.ub)

    # Solve QP
    results = OSQP.solve!(ctrl.solver)
    Δu = results.x[1:7]
    
    k = get_k(ctrl, time)
    return ctrl.Uref[k] + Δu 
end

function lqr(A,B,Q,R; P=Matrix(Q), tol=1e-8, max_iters=400, verbose=false)
    # initialize the output
    n,m = size(B)
    K = zeros(m,n)
    
    # TODO: calculate the infinite-horizon LQR solution
    for k = reverse(1:N-1) 
        K .= (R + B'P*B)\(B'P*A)
        P .= Q + A'P*A - A'P*B*K
    end
    # return the feedback gains and ctg matrices
    return K,P
end

"""
    get_k(ctrl, t)

Get the time index corresponding to time `t`. 
Useful for implementing zero-order hold control.
Uses binary search to find the time index.
"""
get_k(controller, t) = searchsortedlast(controller.times, t)

get_k

In [74]:
# TASK: Implement the following methods:
#       buildQP_unconstrained!  (15 pts)
#       updateQP_unconstrained! (15 pts)
function buildQP_unconstrained!(ctrl::MPCController, A,B,Q,R,Qf; kwargs...)
    # TODO: Implement this method to build the QP matrices
    Nx = n
    Nu = m
    Nh = ctrl.Nmpc - 1
    ctrl.A .= sparse([[B -I zeros(Nx,(Nh-1)*(Nu+Nx))]; zeros(Nx*(Nh-1),Nu) [kron(Diagonal(I,Nh-1), [A B]) zeros((Nh-1)*Nx,Nx)] + [zeros((Nh-1)*Nx,Nx) kron(Diagonal(I,Nh-1),[zeros(Nx,Nu) Diagonal(-I,Nx)])]])
    ctrl.P .= sparse([kron(Diagonal(I,Nh-1),[R zeros(Nu,Nx); zeros(Nx,Nu) Q]) zeros((Nx+Nu)*(Nh-1), Nx+Nu); zeros(Nx+Nu,(Nx+Nu)*(Nh-1)) [R zeros(Nu,Nx); zeros(Nx,Nu) Qf]])
    
    # Initialize the included solver
    #    If you want to use your QP solver, you should write your own
    #    method for this function
    initialize_solver!(ctrl; kwargs...)
    return nothing
end


function updateQP_unconstrained!(ctrl::MPCController, x, time)
    # TODO: Implement this method
    ctrl.ub[1:n] = -A*(x-xeq)
    ctrl.lb[1:n] = -A*(x-xeq)
    
    q = zeros(0)
    k = get_k(ctrl, time)
    Nref = length(ctrl.times)
    
    for i = k:ctrl.Nmpc+k-2
        if i <= Nref - 1
            x_bar = ctrl.Xref[i+1]
            u_bar = ctrl.Uref[i]
        else
            x_bar = ctrl.Xref[Nref]
            u_bar = ctrl.Uref[Nref]
        end
        append!(q,-R*(u_bar - ueq))
        if i != ctrl.Nmpc+k-2
            append!(q,-Q*(x_bar - xeq))
        else
            append!(q,-Qf*(x_bar - xeq))
        end
    end
    ctrl.q .= q
    
    return nothing
end

updateQP_unconstrained! (generic function with 1 method)

In [75]:
# Initialize the unconstrained MPC problem
Nmpc = 21           # MPC Horizon
mpc1 = OSQPController(n, m, Nmpc, length(Xref))
K,Qf = lqr(A,B,Q,R)

# Provide the reference trajectory
mpc1.Xref .= Xref
mpc1.Uref .= Uref
mpc1.times .= tref

# Build the sparse QP matrices
buildQP!(mpc1, A,B,Q,R,Qf, tol=1e-2)

In [76]:
Xmpc1,Umpc1,tmpc1 = simulate(model, Xref[1], mpc1, tf=tf)
println(Xmpc1)
println()
println(Umpc1)

Controller ran at 264.09496047628625 Hz


SVector{14, Float64}[[4.9268856741946365e-5, 0.26016423071338934, 3.140069344584098, -2.27008100955155, -2.40769461674617e-5, 0.9598460188468598, 1.569995694393751, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.0014988238625484689, 0.2600321709657594, 3.143507529620063, -2.269538271572803, 0.00878367964931951, 0.9576975467129473, 1.5649670159644704, -0.30946932903784147, -0.026225113274118264, 0.687536857169352, 0.1092864753630761, 1.7660657852635644, -0.4274064558708594, -1.0104634577331109], [-0.005103997236197301, 0.25971346357321107, 3.1516359529195572, -2.267945789212256, 0.030350803261268315, 0.95162102440542, 1.5541206949753625, -0.41110884603167575, -0.03730360754870245, 0.9379448712691273, 0.21006646995463737, 2.556099475924697, -0.7855874628992072, -1.1673355820209081], [-0.009391890008305002, 0.2593265886076816, 3.1614619825801555, -2.2654131674270293, 0.057772451388618966, 0.9424417318052144, 1.5430957291405503, -0.44589562792524207, -0.03990849877618853, 1.0271606591758644, 0.29

-0.5924893704067123, -0.12028916465505687, 0.3917389603364304, 0.31994139623711254, 2.3422022932847915, 1.2726328609931774, 0.3029296239633819], [-0.4057780223196933, 0.24283744737964988, 3.6610503211291827, -1.9659958825539618, 2.122766489312849, 0.9827170574561523, 2.3418927099649984, -0.5984999798559169, -0.12466548696192721, 0.388140827921965, 0.32067358247072314, 2.2993069792182355, 1.2638564290138974, 0.263744802934467], [-0.4117915910963359, 0.2415687818203127, 3.664913472501157, -1.9627836303327553, 2.1455489859603665, 0.9952981737111538, 2.3443452549309094, -0.6040641086680001, -0.12915918707291604, 0.3844242207775374, 0.3215612189509632, 2.2560683061741904, 1.2522843459265465, 0.22663063785294307], [-0.4178603479458155, 0.2402547404703038, 3.6687416433594717, -1.9595618058415758, 2.167895772259098, 1.0077518173931326, 2.346439026568185, -0.6095574521547473, -0.1337463967652989, 0.38117514754588433, 0.3225693570824748, 2.2120726250708773, 1.2382809883857988, 0.1920700547273843

 3.84524650780867, -1.8307282704811167, 2.630075238777563, 1.2574555653825186, 2.343994524524145, -1.0231220322976682, -0.31181362879122354, 0.8619285977713379, 0.512236726592106, 0.6579035542078944, 0.20947356196461303, -0.026493703760730163], [-0.6843094662395905, 0.16459243233635262, 3.854008765256689, -1.8255574477692558, 2.6365003043230515, 1.2594084012232605, 2.343738894316218, -1.0448965159304953, -0.31795256441716313, 0.8911430470943488, 0.5215693807298477, 0.6260825112788306, 0.18111043552374897, -0.024378045978462732], [-0.6948683072019977, 0.16138264572101096, 3.8630668121070615, -1.8202933759925897, 2.6426157298853585, 1.2610817161462902, 2.3435032349430935, -1.0674553943331357, -0.3242075116413459, 0.9211387447481414, 0.5308834799079202, 0.5960120184340605, 0.1536049810749318, -0.02250686969932669], [-0.7056576391117085, 0.15810970388101253, 3.872429887451533, -1.8149363785504424, 2.6484386074021224, 1.2624842172919701, 2.343285563517442, -1.091061657703047, -0.33059037275

, 3.111543084548079, 1.1536527974580915, 0.3290944679067108, -0.25798947044788284, -0.05704984472214628], [-1.33227693587115, -0.013194131486141692, 4.477424204311733, -1.567657779601154, 2.767921745904672, 1.2102851685603733, 2.334467147807413, -3.1244980002143166, -0.8615993618409267, 3.0272490920388724, 1.2199726565259559, 0.3136081811043007, -0.25025379756841654, -0.05124774518819641], [-1.3654960604477648, -0.02203256699712766, 4.509629129552057, -1.5550706750018453, 2.7710649634195135, 1.2079756553579268, 2.3339864305979665, -3.537609041860832, -0.9087482945033379, 3.4288106584331692, 1.300617139356135, 0.31920685719701686, -0.2029516443751923, -0.04854103170021326], [-1.3988387982694468, -0.031194291781585948, 4.5419257127374415, -1.5417121805682363, 2.774006452673008, 1.2057944743185023, 2.3336197980198095, -3.1636428613063234, -0.9259481734838167, 3.062109581143164, 1.37018753206659, 0.27021394051855663, -0.2338767129881861, -0.025255898422825124], [-1.4343704388068712, -0.040

...

], [0.09326867643375983, -8.74246534449929, 0.6329475555951037, 5.541610694320158, -0.46270277967989926, 0.13605909344946576, -0.029099283401059294], [0.10103154235191415, -8.711979684314464, 0.6383919488291749, 5.55892901899004, -0.45590169027780814, 0.13393586803753577, -0.028555452453447233], [0.10776542266310662, -8.680401797785645, 0.6445009733364185, 5.576076801671966, -0.44921838586562624, 0.132221119004456, -0.028013645659462565], [0.11394472879653991, -8.648046989477725, 0.6507674523001272, 5.593124052925907, -0.4425288634186506, 0.1307042017362493, -0.027477984675686507], [0.11979201390574366, -8.615055901015207, 0.6569853222893332, 5.610096777551497, -0.43579064610294105, 0.12928346378031397, -0.026949966211327565], [0.11488721624650253, -8.582013936139516, 0.6594341486802212, 5.628191268958476, -0.43062752481049227, 0.12906997986207192, -0.026454259736510156], [0.12221609090889383, -8.54874710564514, 0.6640607648536992, 5.645223144841263, -0.42339261571424025, 0.12722259417

, 9.945969264292787, -0.6401908238715512, 6.675163766878186, 0.32000597631851124, -1.449390141935683, 0.1341680654395651], [-3.96207607299657, 11.113753306561785, -1.1683233335829097, 6.7891371553105895, 0.09746159690311329, -1.2849219060464319, 0.15036219423928585], [-5.389825942006218, 11.92634479987961, -1.6956651218737622, 6.8656272301796974, -0.36408277628161945, -1.076468371172842, 0.16780001145620693], [-6.910561790640067, 12.092996322987185, -2.273200050654671, 6.91607241424108, -1.0975547408997193, -0.8920559559265516, 0.18559115762800274], [-8.382110096215994, 11.338380791836057, -2.8366931581397665, 6.983666632897122, -2.02663791655512, -0.7951705586201624, 0.19137703056221075], [-9.540823762249858, 9.554738924209326, -3.025338119449483, 7.18308083113318, -2.680833721719809, -0.7064361764772594, 0.13882866766990065], [-9.951656626882887, 7.3191828174856095, -2.1397638909061207, 7.712659118676792, -2.0677243349037093, -0.40336704269589985, -0.035193772264715796], [-9.41974379

, -3.5184391799664914, 0.14999939449022381], [2.523185792620999, 23.5995368608979, 19.91204485175219, 20.741349982410526, -1.2764318247793836, -2.31708302640199, 0.011396826793562957], [2.2868397324625325, 23.2826693604878, 20.471361380933015, 20.416495654090177, 0.11493049233069674, -1.5957745134793155, -0.05892593159274024], [1.520743346710764, 24.49519573224954, 20.189852814743567, 20.66201808080286, 0.5952220369052211, -1.7563930324975372, -0.026241615659934492], [0.38216366720900996, 26.486556038882675, 19.274656514404175, 20.281237586867846, -0.37432821915442904, -2.571707686968435, 0.08013793192593067], [-1.3841944842250418, 27.75289644404436, 18.013276356252764, 20.464753300403892, -0.2507705302955535, -3.187745409734586, 0.12256203762857204], [-3.641600429648414, 28.685673078972023, 16.362702623063733, 19.98649787258573, -0.99531201652591, -3.8487428289912904, 0.19432306696051044], [-6.735627499691028, 28.623491740534178, 14.284046685990775, 19.95120924062652, -1.1213999422033

, 10.667518654365692, 23.517663370991837, 25.39415229530715, 3.5785290753449743, -8.357360206125735, 0.060640261314001846], [-20.620578521354236, 2.9805505805639214, 21.207611394743825, 27.42901957389033, 4.926434083216359, -8.190291703589445, 0.19699123575611552], [-23.65634913459985, -6.424458557013513, 20.89847848617495, 27.449104762461467, 5.053360070240189, -7.573510003694433, 0.3757149018314598], [-25.108097934115495, -16.173773865215807, 21.188868327082005, 26.142164639356018, 4.0957369624214595, -6.845460634500362, 0.551575881540383], [-26.395929933825087, -26.981043655512806, 20.77902934454906, 23.942235598312926, 2.8503036567908566, -5.950062211986706, 0.6602258747568999], [-28.01212345630422, -40.261767593687146, 19.062052305959572, 20.70501854128041, 1.9389018458738516, -4.815455562278515, 0.684380571648853], [-30.012074678723714, -56.38719644290917, 15.479989625253337, 15.767032439176814, 1.527436166481536, -3.6228563267372778, 0.6741310102368506], [-32.55710892672543, -72

], [-26.097645556103277, 29.260733222590538, 66.37498840561847, 13.528386389260508, 1.1008499419907978, 0.020635250976375752, -0.7171034508742751], [-55.766614457076656, -9.03140018062656, 43.76031288724731, 18.20306488058324, 0.10543200760132798, -5.481654962169842, 0.3498805723588748], [-75.22548192506383, -82.61883131009857, 20.84754716761472, 15.160075815527641, 4.198665789161102, -14.9781748823446, 0.9221783368384402], [-69.3632444801193, -166.08554072018467, 2.5673559135815793, -14.368306433403818, 3.6122828867909496, -20.416489341075813, 1.148103842885386], [-36.66535921219399, -132.15402066341085, 28.698438544041185, -54.65247852506669, -7.123652290187972, -9.452280637397255, 0.2671494651312052], [-7.358869541057373, -38.03662713202687, 59.93812453723618, -30.054630217992667, -9.522897667525127, -15.299643646782298, 0.2315454299006149], [-0.8421164483086401, 30.108425846170157, 72.19228524423255, 0.6366640757914315, -0.6880827763099384, -20.051958733086668, -0.04426236105820619

In [64]:
using MeshCat, MeshCatMechanisms, Blink
urdf = "../../../kortex_description/arms/gen3/7dof/urdf/GEN3_URDF_V12 copy.urdf"
body = findbody(model.mechanism, "Bracelet_Link")
point = Point3D(default_frame(body), 0., 0, -0.07)
# Create the visualizer
vis = MechanismVisualizer(model.mechanism, URDFVisuals(urdf))

# Render our target point attached to the robot as a sphere with radius 0.07
setelement!(vis, point, 0.01)
render(vis)
# mvis = MechanismVisualizer(model.mechanism, URDFVisuals(urdf))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /home/amkyu/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Shoulder_Link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x6c, 0x64, 0x65, 0x72, 0x5f, 0x4c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Actuator1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x41, 0x63, 0x74, 0x75, 0x61, 0x74, 0x6f, 0x72, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "HalfArm1_Link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x41, 0x72, 0x6d, 0x31, 0x5f, 0x4c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("HalfArm2_Link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x41, 0x72, 0x6d, 0x32, 0x5f, 0x4c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("ForeArm_Link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x65, 0x41, 0x72, 0x6d, 0x5f, 0x4c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Actuator4" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x41, 0x63, 0x74, 0x75, 0x61, 0x74, 0x6f, 0x72, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "SphericalWrist1_Link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x69, 0x73, 0x74, 0x31, 0x5f, 0x4c, 0x69, 0x6e, 0x6b], Dict

In [77]:
qs = Vector{Float64}[]
for x in Xmpc1
    push!(qs, copy(x[1:7]))
end
ts = tmpc1;

In [78]:
setanimation!(vis, Animation(vis, ts, qs))

In [ ]:
# TODO:
# Uref could be just U equilibrium (just gravity compensation),
# Xref could be the desired joint and cartesian positions and velocities, and then 0 for the forces
state = MechanismState(model.mechanism)
v̇ = similar(velocity(state))
for joint in joints(state.mechanism)
    if joint != joints(state.mechanism)[1] && joint != joints(state.mechanism)[end]
        v̇[joint][1] = 0.0
    end
end

Uref = typeof(@SVector zeros(7))[]
Xref = typeof(x0)[]
push!(Xref, copy(x0))

for k in 1:N-1
    q = q0 + k*(qf - q0)/(N-1)
    if k != N-1
        q̇ = (qf - q0)/(N-1)/dt
    else
        q̇ = @SVector zeros(7)
    end
    
    x = [q; q̇]
    push!(Xref, copy(x))
    
    zero!(state)
    set_configuration!(state, q)
    set_velocity!(state, q̇)
    uTest = inverse_dynamics(state, v̇)
    push!(Uref, copy(SVector{7, Float64}(uTest.parent)))
end

dtref = [dt for k=1:N]
traj = RobotDynamics.Traj(Xref, Uref, dtref, cumsum(dtref) .- dtref[1]);

In [ ]:
# Set up
# TODO: Tune weights
Q = 1.0e-2*Diagonal(@SVector ones(n))
Qf = 100*Diagonal(@SVector ones(n))
R = 1.0e-1*Diagonal(@SVector ones(m))
obj = TrajectoryOptimization.TrackingObjective(Q, R, traj, Qf=Qf)

In [ ]:
# Create Empty ConstraintList
conSet = ConstraintList(n,m,N)

# Control Bounds based on Robot Specs (Joint torque limits)
u_bnd = [39.0, 39.0, 39.0, 39.0, 9.0, 9.0, 9.0]
control_bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
add_constraint!(conSet, control_bnd, 1:N-1)

# State Bounds based on Robot Specs (Joint velocity and speed limits)
x_bnd = zeros(n)
x_bnd[1:7] = [Inf, deg2rad(128.9), Inf, deg2rad(147.8), Inf, deg2rad(120.3), Inf] # rad
x_bnd[8:14] = [1.39, 1.39, 1.39, 1.39, 1.22, 1.22, 1.22] # rad/sec
# x_bnd[15:end] = [Inf for k=1:(n-14)] # Constraints on force elsewhere
state_bnd = BoundConstraint(n,m, x_min=-x_bnd, x_max=x_bnd)
add_constraint!(conSet, state_bnd, 1:N)

# # Cartesian Velocity Bound
# ẋ_max = 0.0005 # m/s
# vel_bnd = NormConstraint(n, m, ẋ_max, Inequality(), 21:23)
# add_constraint!(conSet, vel_bnd, 1:N)

# # Force Bound (Fx Fy Fz)
# F_max = 20 # Newtons
# F_bnd = NormConstraint(n, m, F_max, Inequality(), 27:29)
# add_constraint!(conSet, F_bnd, 1:N)

# Other possible constraints:
# Cartesian angular velocity bounds?
# External moment bounds?

# Goal Constraint - only if you want the final state to be the desired state
# goal = GoalConstraint(xf)
# add_constraint!(conSet, goal, N)

In [ ]:
prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet, X0=Xref, U0=Uref)

In [ ]:
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0,
)

altro = ALTROSolver(prob, opts)
set_options!(altro, show_summary=true)
@time solve!(altro)

In [ ]:
# Get some info on the solve
max_violation(altro)
cost(altro)
iterations(altro)

# Extract the solution
X = states(altro)
U = controls(altro)

# Extract the solver statistics
stats = Altro.stats(altro)   # alternatively, solver.stats
stats.iterations             # equivalent to iterations(solver)
stats.iterations_outer       # (number of Augmented Lagrangian iterations)
stats.iterations_pn          # (number of projected newton iterations)
stats.cost[end]              # terminal cost
stats.c_max[end]             # terminal constraint satisfaction
stats.gradient[end]          # terminal gradient of the Lagrangian
dstats = Dict(stats)         # get the per-iteration stats as a dictionary (can be converted to DataFrame)

In [ ]:
for i in 1:N
    print("x")
    print(i-1)
    print(": ")
    println(X[i])
end
println()

for i in 1:N-1
    print("u")
    print(i)
    print(": ")
    println(U[i])
end
println()
print("Terminal Error: ")
println(norm(X[end] - xf))
println(X[end] - xf)

In [ ]:
XTest = Vector{Float64}[]
UTest = Vector{Float64}[]
push!(XTest, copy(x0))
state = MechanismState(model.mechanism)
v̇ = similar(velocity(state))
# for joint in joints(state.mechanism)
#     if joint != joints(state.mechanism)[1] && joint != joints(state.mechanism)[end]
#         v̇[joint][1] = -0.1

#     end
# end

for k = 1:N-1
    i = 8
    for joint in joints(state.mechanism)
        if joint != joints(state.mechanism)[1] && joint != joints(state.mechanism)[end]
            v̇[joint][1] = Xref[k+1][i]
            i = i + 1
        end
    end
    
    zero!(state)
    set_configuration!(state, XTest[k][1:7])
    set_velocity!(state, XTest[k][8:end])
    uTest = inverse_dynamics(state, v̇)
    push!(UTest, copy(uTest))
#     println(uTest)
    
    dx = dynamics(model, XTest[k], uTest)
#     dx = dynamics(model, XTest[k], U[k])
    push!(XTest, copy(XTest[k] + dx*dt))
end
# println(XTest)

println(UTest)

println((U - UTest))

In [ ]:
qs = Vector{Float64}[]
for x in XTest
    push!(qs, copy(x[1:7]))
end
ts = collect(0:dt:(N-1)*dt);

In [ ]:
setanimation!(vis, Animation(vis, ts, qs))

In [ ]:
# function rpyxyz2H(xyzrpy)
#     xyz = xyzrpy[1:3]
#     rpy = xyzrpy[4:6]
#     Ht = [[1, 0, 0, xyz[1]] [0, 1, 0, xyz[2]] [0, 0, 1, xyz[3]] [0, 0, 0, 1]]
#     Hx = [[1, 0, 0, 0] [0, cos(rpy[1]), -sin(rpy[1]), 0] [0, sin(rpy[1]), cos(rpy[1]), 0] [0, 0, 0, 1]]
#     Hy = [[cos(rpy[2]), 0, sin(rpy[2]), 0] [0, 1, 0, 0] [-sin(rpy[2]), 0, cos(rpy[2]), 0] [0, 0, 0, 1]]
#     Hz = [[cos(rpy[3]), -sin(rpy[3]), 0, 0] [sin(rpy[3]), cos(rpy[3]), 0, 0] [0, 0, 1, 0] [0, 0, 0, 1]]
#     H = Ht*Hz*Hy*Hx
#     return H
# end

In [ ]:
# using PyCall
# pushfirst!(pyimport("sys")."path", "")
# ArthurBot = pyimport("ArthurBot")
# art = ArthurBot.ArthurBot()
# pycall(art.IterInvKin, PyAny, art, qInit, rpyxyz2H(ps[:,1]))
# x="IterInvKin(qInit, rpyxyz2H(ps[:,1]))"
# py"ArthurBot.IterInvKin"(qInit, rpyxyz2H(ps[:,1]))
# pycall(function::art.IterInvKin, returntype::PyAny, qInit, rpyxyz2H(ps[:,1]))
# art.IterInvKin(art, qInit, rpyxyz2H(ps[:,1]))
# art.